In [1]:
import re
from collections import Counter
from typing import List, Tuple

from pymorphy2 import MorphAnalyzer
# from bedarev_analyzer import MorphAnalyzer

In [2]:
with open("./data/clear_text.txt", 'r', encoding='utf8') as fp:
    text = fp.read()

In [2]:
class NGrammer():
    morph = MorphAnalyzer()
    DELIMITERS = (',', '.', '!', '?', ':', ';')
    
    def __init__(self, text, min_count = 2):
        for delimiter in self.DELIMITERS:
            text = text.replace(delimiter, '\n')
        normal_text = [l for l in (self.normalize(line) for line in text.split('\n')) if l]
        self.min_count = max(2, min_count)
        self.dicts_all_n_gramms = []

        for n in range(1, max([len(l) for l in normal_text]) + 1):
            all_n_gramms = []
            for normal_line in normal_text:
                if len(normal_line) < n:
                    continue
                n_gramms = self.get_n_gramm(normal_line, n)
                if n_gramms:
                        all_n_gramms.extend(n_gramms)
            dict_all_n_gramms = {name: count for name, count in Counter(all_n_gramms).items() if name and count >= self.min_count}
            if not dict_all_n_gramms:
                break
            self.dicts_all_n_gramms.append(dict_all_n_gramms)

    @classmethod
    def normalize(cls, line: str) -> List[str]:
        return [w for w in [cls.morph.parse(word)[0].normal_form for word in
                (v.lower() for v in re.findall(r"('?[а-яА-ЯёЁ][а-яА-ЯёЁ]*(?:-[а-яА-ЯёЁ]+)*'?)", line))] if w and w != ' ']

    @staticmethod
    def get_n_gramm(words: list, n: int = 2): 
        return [tuple(words[i-n+1:i+1]) for i in range(n - 1, len(words))]

    def max_n(self):
        return len(self.dicts_all_n_gramms) - 1

    def gramms(self, n=None):
        result = Counter()
        if n is None:
            for i in self.dicts_all_n_gramms[1:]:
                result += Counter(i)
        else:
            result = Counter(self.dicts_all_n_gramms[n-1])
        result = Counter({' '.join(key): value for key, value in result.items()})
        return result

In [4]:
%%time
grammer = NGrammer(text)

Wall time: 1min 44s


In [5]:
len(grammer.gramms())

264860

In [6]:
print(len(grammer.gramms(2)))
grammer.gramms(2).most_common(5)

55089


[('в работа', 871),
 ('в год', 757),
 ('на основа', 485),
 ('один из', 439),
 ('а также', 423)]

In [7]:
print(len(grammer.gramms(3)))
grammer.gramms(3).most_common(5)

44980


[('в тот число', 175),
 ('в дать работа', 161),
 ('в настоящее время', 157),
 ('в связь с', 151),
 ('по сравнение с', 148)]

In [8]:
grammer.gramms(4).most_common(5)

[('в то же время', 63),
 ('в связь с это', 56),
 ('в то время как', 53),
 ('один и тот же', 37),
 ('в тот число и', 31)]

In [9]:
print(grammer.max_n())
grammer.gramms(grammer.max_n())

40


Counter({'в работа быть сформулировать модель электрический инициирование пентаэритриттетранитрат тэн в канал детонатор и быть провести численный анализ развитие детонация для разный плотность век и для разный энергия инициирование на основа моделирование распространение ударный волна в цилиндрический канал по двумерный газодинамический': 2,
         'работа быть сформулировать модель электрический инициирование пентаэритриттетранитрат тэн в канал детонатор и быть провести численный анализ развитие детонация для разный плотность век и для разный энергия инициирование на основа моделирование распространение ударный волна в цилиндрический канал по двумерный газодинамический программа': 2})